## Tutorial 2 smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

In [1]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

### Cargar la clave de hugging face desde el fichero .env

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel(model_id=model_id)

In [3]:
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id='huggingface/'+model_id)

e:\JAZ\AI3\ACADEMY\RETOSMOLAGENT\SMOL_AGENT_FERRETERIA\JAZ\EJEMPLO1\.VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Utilidades

### 1. Buscar en internet conDuckDuckGo

In [ ]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))

### 2. Crear una tool para obtener el título de una página web

La siguiente versión a mi no me funciona:

In [ ]:
from smolagents import ToolCallingAgent

agent = ToolCallingAgent(tools=[], model=model)
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

Pero si se define como una tool, entonces si funciona:

In [ ]:
!pip install bs4 BeautifulSoup4

In [4]:
import requests
from bs4 import BeautifulSoup
from smolagents import tool, CodeAgent, HfApiModel

# Definir la función para obtener el título de la página web
@tool
def get_title(url: str) -> str:
    '''
    Obtener el título de una página web

    Args:
        url: Url de la página web
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_tag = soup.find('title')
    return title_tag.text if title_tag else "No title found"

# Crear la lista de herramientas con la función de obtener título
tools = [
    {
        "name": "get_title",
        "function": get_title,
        "description": "Fetches the title of a webpage given a URL."
    }
]

# you can also specify a particular provider e.g. provider="together" or provider="sambanova"
agent = CodeAgent(tools=[get_title], model=model, add_base_tools=True)

In [6]:
prompt_templates = agent.prompt_templates
agent.prompt_templates['system_prompt'] += "\nWhen you generate a final answer, you must stop immediately."

In [7]:
# Ejecutar el agente con la solicitud
result = agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog'?                                    │
│                                                                                                                 │
╰─ LiteLLMModel - huggingface/meta-llama/Llama-3.3-70B-Instruct ──────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  title = get_title(url='https://huggingface.co/blog')                                                             
  print(title)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Hugging Face – Blog

Out: None

[Step 0: Duration 4.71 seconds| Input tokens: 23 | Output tokens: 67]

...

Out: {'type': 'function', 'name': 'final_answer', 'parameters': {'answer': {'type': 'any', 'value': 'Hugging Face –
Blog'}}}

[Step 5: Duration 2.74 seconds| Input tokens: 269 | Output tokens: 247]

Reached max steps.

[Step 6: Duration 2.74 seconds| Input tokens: 359 | Output tokens: 272]

The title of the page at the URL 'https://huggingface.co/blog' is 'Hugging Face – Blog'.
